In [62]:
import pennylane as qml
from pennylane import numpy as np
from IPython.display import display, Latex

# from numba import jit

# numpy 
import numpy as np_
import scipy as sp

import matplotlib.pyplot as plt

# 2-qubit
dev = qml.device('default.mixed', wires = 2)

# == Hamiltonian setup == 
coeffs = [1]
obs = [qml.PauliZ(0) @ qml.PauliZ(1)]

hamiltonian = qml.Hamiltonian(coeffs, obs)

In [63]:
# == Global paras ==
Paras_global, Phi_global, Gamma_ps  = np_.zeros(6), 0, 0

# Return one by one
def Dephase_factor(tau):
    Calculate_Dephase = 1 - np.exp(-2 * tau)
    
    return Calculate_Dephase

$$
\rho_{After-Hadamard} = 

\frac{1}{2} 

\begin{bmatrix}

1 & e^{(i\phi - \tau)} \\
e^{(-i\phi - \tau)} & 1

\end{bmatrix}

=

\frac{1}{2} 

\begin{bmatrix}

1 & e^{i\phi} \sqrt{1 - \gamma} \\
e^{-i\phi} \sqrt{1 - \gamma} & 1

\end{bmatrix}$$

$$

where,
\quad \gamma = 
1 - e^{-2 \tau}
$$


$$ 
e^{-\tau} = \sqrt{1 - \gamma}
$$

In [64]:
# == Generate circuit ==

# circuit_1
@qml.qnode(dev)
def circuit_1(paras):
    # Call global phi
    global Phi_global
    phi = Phi_global
    
    # Assign paras
    theta_init, tau_1, tau_2 = paras[0], paras[1], paras[2]
    
    # Calculate dephase factor with given paras
    gamma_dephase_in = np_.zeros(3)
    for i in range(3):
        gamma_dephase_in[i] = Dephase_factor(paras[i+3])
        
        
    # Stage_1: RY for pi/2
    qml.RY(np.pi/2, wires=0)
    qml.RY(np.pi/2, wires=1)
    
    # Stage_2: Entangler    
    qml.ApproxTimeEvolution(hamiltonian, tau_1, 1)
    qml.PhaseDamping(gamma_dephase_in[0], wires = 0)
    qml.PhaseDamping(gamma_dephase_in[0], wires = 1)
    
    qml.RX(theta_init, wires = 0)    
    qml.RX(theta_init, wires = 1)    

    qml.RY(-np.pi/2, wires = 0)    
    qml.RY(-np.pi/2, wires = 1)   

    qml.ApproxTimeEvolution(hamiltonian, tau_2, 1)
    qml.PhaseDamping(gamma_dephase_in[1], wires = 0)
    qml.PhaseDamping(gamma_dephase_in[1], wires = 1)

    qml.RY(np.pi/2, wires = 0)    
    qml.RY(np.pi/2, wires = 1) 
    
    # Stage_3: Accumulater
    qml.ApproxTimeEvolution(hamiltonian, phi, 1)
    qml.PhaseDamping(gamma_dephase_in[2], wires = 0)
    qml.PhaseDamping(gamma_dephase_in[2], wires = 1)
    
    qml.RY(-np.pi/2, wires=0)
    qml.RY(-np.pi/2, wires=1)
    
    return qml.density_matrix(wires=[0, 1])


# Circuit_2: Calculate decoherence by using [tau -> gamma_dephase_in] conversion
@qml.qnode(dev)
def Post_selection_Dephase(phi):
    
    # theta_init, tau_1, tau_2, gamma_1, gamma_2, gamma_3 = paras[0], paras[1], paras[2], paras[3], paras[4], paras[5]
    global Paras_global, Phi_global, Gamma_ps
    Phi_global = phi
    
    # Get density matrix from circuit_1
    density_matrix = circuit_1(Paras_global)
    qml.QubitDensityMatrix(density_matrix, wires=[0, 1])
    
    # Kraus operator for 2*2 matrix
    K = np.array([ [np.sqrt(1 - Gamma_ps), 0], [0, 1] ])
    K_H = K.conj().T
    
    Numerator = sp.linalg.kron(K, K) @ density_matrix @ sp.linalg.kron(K_H, K_H)
    Denominator = np.trace(sp.linalg.kron(K, K) @ density_matrix @ sp.linalg.kron(K_H, K_H))
    
    rho_ps = Numerator / Denominator
    
    qml.QubitDensityMatrix(rho_ps, wires=[0, 1])
    
    return qml.density_matrix(wires=[0, 1])  

$$ K =  \begin{bmatrix}
\sqrt{1-\gamma} & 0 \\
0 & 1 
\end{bmatrix}  $$


$$
\rho_{ps} = 
\frac{(K \otimes K) \rho (K^{\dagger} \otimes K ^{\dagger})}

{Tr[(K \otimes K) \rho (K^{\dagger} \otimes K ^{\dagger})]}
$$

![Alt text](image-1.png)

In [65]:
# == Cost function ==
def Cost_function(paras):
    # expect: theta_init, tau_1, tau_2, tau_d1, tau_d2, tau_d3 = paras[0], paras[1], paras[2], paras[3], paras[4], paras[5]
    global Paras_global, Phi_global
    Paras_global, phi = paras, np.array([Phi_global])
          
    CFI = qml.qinfo.classical_fisher(Post_selection_Dephase)(phi[0])
    
    return -CFI

In [66]:
# == Check for maximum tau <-> gamma ==
tau_test = 18

print('Gamma =', Dephase_factor(tau_test), ', [ at tau =',tau_test,']' )

Gamma = 0.9999999999999998 , [ at tau = 18 ]


In [67]:
# == SLSQP -> Return Data_set:[phi, CFI, 6-Paras] ==
def SLSQP_test(Sweep_Data, paras, gamma_ps_select):
    # Create Data array
    PHI = np_.arange(Sweep_Data[0], Sweep_Data[1], Sweep_Data[2]).reshape(-1, 1)
    
    Data = np_.zeros((len(PHI), len(paras) + 1))
    Data = np_.hstack((PHI, Data))
    
    # Set Data index
    CFI_INDEX, PARAS_INDEX = 1, 2
    
    # Set global variables
    global Gamma_ps, Phi_global, Paras_global
    Gamma_ps = gamma_ps_select
    
    # Optimize begin
    for i in range(len(PHI)):
        # Assign global phi from Generated PHI array
        Phi_global = Data[i][0]
        
        # Assign global paras for each phi
        
        # Set constraints for gamma_from_tau
        tau_upper = 18
        Constraints = [(-float('inf'), float('inf'))] * 3 + [(1e-100, tau_upper)] * 3
        
        Paras_global = paras 
        Result_SLSQP = sp.optimize.minimize(Cost_function, paras, method = 'SLSQP', bounds = Constraints)
        Data[i][CFI_INDEX] = -Result_SLSQP.fun
        
        # Store data in array
        for j in range(len(paras)):
            Data[i][PARAS_INDEX + j] = Result_SLSQP.x[j]
                
    return Data

In [68]:
# == Fixed to gamma_ps = 0.8 ==
# == Sweep for 0 ~ pi*3 == 
Sweep_data = np_.array([1e-3, np.pi /8 + 1e-3, 1e-1])
# Para_init = np_.array([1, 1, 1, 1e-1, 1e-1, 1e-1])

Para_init = np.array([-3.04130343e+01, -5.33930331e+01, 1.00062174e+00,  2.83292962e-02,  1.23605402e+00,  2.84133704e-02])
       
Gamma_ps_preset = 0.8

Optimized_Result_test = SLSQP_test(Sweep_data, Para_init, Gamma_ps_preset)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


In [69]:
Optimized_Result_test[3][:]

array([ 3.01000000e-01,  5.64390898e+01, -3.46473823e+01, -5.36194129e+01,
        1.00123868e+00,  2.39967032e-05,  2.58237192e-05,  2.39967285e-05])

Data = 

array([ [PHI_0, CFI_0, theta_0, tau_1, tau_2, tau_d1, tau_d2, tau_d3], 

...

In [70]:
# == Fixed to gamma_ps = 0.8 ==
# == Sweep for 0 ~ pi*3 == 
Sweep_data = np_.array([1e-3, np.pi /8 + 1e-3, 1e-1])
# Para_init = np_.array([1, 1, 1, 1e-1, 1e-1, 1e-1])

Para_init = np.array([-3.04130343e+01, -5.33930331e+01, 1.00062174e+00,  2.83292962e-02,  1.23605402e+00,  2.84133704e-02])
       
Gamma_ps_preset = 0.8

Optimized_Result_test = SLSQP_test(Sweep_data, Para_init, Gamma_ps_preset)

In [71]:
Optimized_Result_test[3][:]

array([ 3.01000000e-01,  5.64390898e+01, -3.46473823e+01, -5.36194129e+01,
        1.00123868e+00,  2.39967032e-05,  2.58237192e-05,  2.39967285e-05])

In [72]:
for i in range(Iterations_set):
    plt.plot(Optimized_Result_test[0, :, 0], Optimized_Result_test[i, :, 1], label = f'Iterations = {i}')
    
plt.title('CFI After SLSQP')
plt.xlabel('Time')
plt.ylabel('CFI')
plt.legend()
plt.grid() 

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

Each row of 6-paras => Corresponds to phi

func(phi, paras, gamma_ps)
=> Implement on SLSQP 

Data = [[phi, CFI, theata_init, tau_1, tau_2, tau_d1, tau_d2, tau_d3] ... N]

In [80]:
dd = np_.array([0,1,2,3,4,5,6,7])

Data = np_.zeros((2, len(dd), 3))

Data[:,:,0] = dd

Data

array([[[0., 0., 0.],
        [1., 0., 0.],
        [2., 0., 0.],
        [3., 0., 0.],
        [4., 0., 0.],
        [5., 0., 0.],
        [6., 0., 0.],
        [7., 0., 0.]],

       [[0., 0., 0.],
        [1., 0., 0.],
        [2., 0., 0.],
        [3., 0., 0.],
        [4., 0., 0.],
        [5., 0., 0.],
        [6., 0., 0.],
        [7., 0., 0.]]])